In [1]:
import os
import re
import json
import gzip
import time
import calendar
from datetime import datetime
import zipfile
import requests
import numpy as np
import pandas as pd
from datetime import date
from decimal import Decimal
import warnings
warnings.filterwarnings('ignore')
CACHE = {} # глобальный кэш, в процессе скрейпинга не обновлять!
SETS = './sets/' # папка с файлами с наборами

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_cache(filename='cache.gz'): # загрузка кэша
    global CACHE
    if os.path.exists(filename) and len(CACHE)==0:
        with gzip.open(filename, 'rb') as gzip_ref:
            CACHE = json.load(gzip_ref)
        print(f'Загружено {len(CACHE)} записей из {filename} в кэш')


def save_cache(filename='cache.gz'): # сохранение кэша
    global CACHE
    with gzip.open(filename, 'wb') as gzip_file:
        json_data = json.dumps(CACHE, ensure_ascii=False, default=default_serializer)
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(CACHE)} записей в {filename}')
    
    
def load_gzon(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return json.load(gzip_ref)
    else:
        return []
    

def save_gzon(filename):
    data = globals().get(filename)
    if data is not None:
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(json_data)} записей в {filename}.gz')
    else:
        print(f'Переменная {filename} не найдена')
        
     
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        
def sets_intersects(exams_set, start, end):
    """
    Для уменьшения объема датасета ecology, проводим проверку, входят ли даты событий в наборы дат экзаменов
    """
    #date_start = datetime.strptime(start, '%d.%m.%Y')
    #date_end = datetime.strptime(end, '%d.%m.%Y')
    date_range = pd.date_range(start=start, end=end)
    date_set = set(date_range)
    if date_set.intersection(exams_set):
        return True
    else:
        return False

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
ecology = load_dataset('ecology')

In [4]:
ecology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189165 entries, 0 to 189164
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   start            189165 non-null  object
 1   end              189165 non-null  object
 2   air              291 non-null     object
 3   mean             903 non-null     object
 4   noise            3418 non-null    object
 5   dirt             124 non-null     object
 6   digging          6740 non-null    object
 7   capital          3490 non-null    object
 8   house_lines      31582 non-null   object
 9   clinics          280 non-null     object
 10  roads            3842 non-null    object
 11  parks            37 non-null      object
 12  transports       108 non-null     object
 13  city_places      3093 non-null    object
 14  live_area        103092 non-null  object
 15  streets          24 non-null      object
 16  trashes          26820 non-null   object
 17  factories 

In [5]:
# ищем записи, где не пустые поля имеют размер менее 4 символов
ecology[(ecology[ecology.columns[2:]].astype(str).applymap(len) < 5).all(axis=1)]

,start,end,air,mean,noise,dirt,digging,capital,house_lines,clinics,roads,parks,transports,city_places,live_area,streets,trashes,factories,markets,autostations,gasstations,bad_gasstations,stroi
56730,26.05.2017,25.08.2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96152,17.06.2019,11.07.2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111588,15.04.2021,03.05.2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140554,01.06.2023,18.08.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151109,01.05.2023,01.05.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151110,01.05.2023,01.05.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151111,01.05.2023,01.05.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151112,01.05.2023,01.05.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151113,01.05.2023,01.05.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151114,01.05.2023,01.05.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# удаляем найденные выше записи
ecology = ecology[~(ecology[ecology.columns[2:]].astype(str).applymap(len) < 5).all(axis=1)]

In [7]:
ecology['start'] = ecology.apply(lambda x: "01.01." + x['start'] if len(x['start']) < 5 else x['start'], axis=1)

In [8]:
ecology['start'] = ecology.apply(lambda x: "01." + x['start'] if len(x['start']) < 8 else x['start'], axis=1)

In [9]:
ecology['end'] = ecology.apply(lambda x: "31.12." + x['end'] if len(x['end']) < 5 else x['end'], axis=1)

In [10]:
ecology['end'] = ecology.apply(lambda x: "28." + x['end'] if len(x['end']) < 8 else x['end'], axis=1)

In [11]:
ecology['start'] = pd.to_datetime(ecology['start'], format='%d.%m.%Y')

In [12]:
%%time
for index, row in ecology.iterrows():
    try:
        ecology.loc[index, 'end'] = pd.to_datetime(row['end'], format='%d.%m.%Y')
    except ValueError:
        print(f"Ошибка на строке {index}: {row}")
        day = min(calendar.monthrange(int(row['end'].split('.')[2]), int(row['end'].split('.')[1]))[1], 28)
        corrected_date = f"{day:02d}.{row['end'].split('.')[1]}.{row['end'].split('.')[2]}"
        ecology.loc[index, 'end'] = pd.to_datetime(corrected_date, format='%d.%m.%Y')

CPU times: user 1min 22s, sys: 41.7 ms, total: 1min 22s
Wall time: 1min 23s


In [13]:
exams = load_dataset('exams')

In [14]:
exams['start'] = exams.apply(lambda x: '01.03.' + x['YEAR'].split('-')[1], axis=1)

In [15]:
exams['end'] = exams.apply(lambda x: '30.09.' + x['YEAR'].split('-')[1], axis=1)

In [16]:
start_list = exams['start'].value_counts()
end_list = exams['end'].value_counts()

In [17]:
%%time
exams_ranges = list(zip(list(start_list.index), list(end_list.index)))
exams_set = set()
for i in exams_ranges:
    start_date = datetime.strptime(i[0], '%d.%m.%Y')
    end_date = datetime.strptime(i[1], '%d.%m.%Y')
    date_range = pd.date_range(start=start_date, end=end_date)
    exams_set |= set(date_range)

CPU times: user 4.87 ms, sys: 0 ns, total: 4.87 ms
Wall time: 6.36 ms


In [18]:
%%time
ecology['exams_period'] = ecology.apply(lambda x: sets_intersects(exams_set, x['start'], x['end']), axis=1)

CPU times: user 2min 34s, sys: 2.17 s, total: 2min 36s
Wall time: 2min 36s


In [19]:
ecology = ecology[ecology['exams_period'] == True]

In [20]:
ecology['start'] = pd.to_datetime(ecology['start'], format='%Y-%m-%d')
ecology['end'] = pd.to_datetime(ecology['end'], format='%Y-%m-%d')
exams['start'] = pd.to_datetime(exams['start'], format='%d.%m.%Y')
exams['end'] = pd.to_datetime(exams['end'], format='%d.%m.%Y')

In [21]:
exams['start'] = pd.to_datetime(exams['start'], format='%Y-%m-%d')
exams['end'] = pd.to_datetime(exams['end'], format='%Y-%m-%d')

In [22]:
end_date =  datetime.strptime("2017-03-01", '%Y-%m-%d')
ecology = ecology[ecology['end'] > end_date]
ecology.drop('exams_period', axis=1, inplace=True)

In [23]:
save_dataset('ecology')
save_dataset('exams')

Сохранено 171715 записей в ecology.gz
Сохранено 5542 записей в exams.gz
